<a href="https://colab.research.google.com/github/navkamboj/INSE6260recommender/blob/main/RestaurantCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data=pd.read_csv('https://raw.githubusercontent.com/navkamboj/INSE6260recommender/main/revisited_restaurants_data.csv')
data.head()
df=pd.read_csv('https://raw.githubusercontent.com/navkamboj/INSE6260recommender/main/convertcsv.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   userID          1161 non-null   object 
 1   placeID         1161 non-null   int64  
 2   overall_rating  1161 non-null   float64
 3   food_rating     1161 non-null   int64  
 4   service_rating  1161 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 45.5+ KB


In [ ]:
def matrix():
    matrix_data=data.pivot_table(index='userID', columns='placeID' , values='overall_rating')
    return matrix_data

In [ ]:
def matrixNormalization():
    matrix_list=matrix()
    matrix_norm=matrix_list.subtract(matrix_list.mean(axis=1),axis='rows')
    return matrix_norm

In [ ]:
# User Similarity matrix using Pearson's correlation
def pearsonCorrelation():
    matrix_norm = matrixNormalization()
    user_similarity=matrix_norm.T.corr()
    return user_similarity

In [ ]:
def recommendRestaurants(picked_userID, number_of_similar_user,number_of_top_restaurants):
    pr=pearsonCorrelation()
    picked_userID= picked_userID
    #Remove picked user Id from the lsit
    pr.drop(index=picked_userID,inplace=True)

    # Number of similar users
    n= number_of_similar_user
    #User similarity threshold
    user_similarity_threshold=0.3
    #Get top n similar users
    similar_users = pr[pr[picked_userID]>user_similarity_threshold][picked_userID].sort_values(ascending=False)[:n]

    #Restaurant visited by the target user
    matrix_normalization= matrixNormalization()
    picked_userID_visited = matrix_normalization[ matrix_normalization.index == picked_userID].dropna(axis=1, how='all')

    #Restaurant that similar user visited. Remove restaurants that none of the similar user have visited   
    similar_user_visits = matrix_normalization[matrix_normalization.index.isin(similar_users.index)].dropna(axis=1, how='all')

    # Remove visited restaurant from the list
    similar_user_visits.drop(picked_userID_visited.columns,axis=1, inplace=True, errors='ignore')

    # A dictionary to store item scores
    item_score = {}

    # Loop through items
    for i in similar_user_visits.columns:
        # Get the ratings for restaurant i
        restaurant_rating = similar_user_visits[i]
        # Create a variable to store the score
        total = 0
        # Create a variable to store the number of scores
        count = 0
         # Loop through similar users
        for u in similar_users.index:
            # If the restaurant has rating
            if pd.isna(restaurant_rating[u]) == False:
                # Score is the sum of user similarity score multiply by the restaurant rating
                score = similar_users[u] * restaurant_rating[u]
                # Add the score to the total score for the restaurant so far
                total += score
                # Add 1 to the count
                count +=1
        # Get the average score for the item
        item_score[i] = total / count

    # Convert dictionary to pandas dataframe
    item_score = pd.DataFrame(item_score.items(), columns=['place', 'place_score'])
        
    # Sort the restaurant by score
    ranked_item_score = item_score.sort_values(by='place_score', ascending=False)

    # Select top m restaurant
    m = number_of_top_restaurants
    return ranked_item_score.head(m)

In [ ]:
def recommendPopularRestaurants():
    unique_place_list=df.placeID.unique().tolist()
    final_list= pd.DataFrame(columns=['placeID','overall_rating'])
    for place in unique_place_list:
        place_exists=data.loc[data['placeID'] == place]
        # drop the columns food, service rating
        place_exists = place_exists.drop(['food_rating', 'service_rating'], axis=1);
        if (len(place_exists.index)>0):
            # print(place)
            new_row = pd.DataFrame(place_exists.groupby('placeID', as_index=False).mean())
            # print(new_row)
            final_list = pd.concat([new_row,final_list.loc[:]]).reset_index(drop=True)
            # print(final_list)
            # final_list=pd.concat([final_list, pd.DataFrame(place_exists.groupby('placeID')['overall_rating'].mean())])
    # print(final_list)
    return final_list

In [ ]:
picked_userID='U1001'
specificUserData=data.loc[data['userID'] == picked_userID]
print(specificUserData)
if (len(specificUserData.index)>2):  
    number_of_similar_user=100
    number_of_top_restaurants=100
    recommendList= recommendRestaurants(picked_userID,number_of_similar_user,number_of_top_restaurants)
    df1 = recommendList.rename(columns={'place':'placeID'})
    joined_frame = df1.merge(df, on='placeID', how='left')
    #Drop N/A
    joined_frame_without_NAN=joined_frame.dropna()
    result=joined_frame_without_NAN.drop('id', axis=1)
    print(result.to_json(orient='records', indent=2))
else:
    pRestaurants = recommendPopularRestaurants()
    # join with restaurants
    popCopyRestaurants = df.copy()
    # print(popCopyRestaurants)
    restaurants =  popCopyRestaurants.merge(pRestaurants, on="placeID", how="right")
    # sort values by asc
    restaurants = restaurants.sort_values(by='overall_rating', ascending=False)
    # drop the id column
    restaurants = restaurants.drop(['id'], axis=1)
    # convert to json response
    print(restaurants.to_json(orient='records', indent=2))

    userID  placeID  overall_rating  food_rating  service_rating
215  U1001   132830             1.0            1               1
216  U1001   132825             1.5            2               1
688  U1001   135085             1.0            1               1
689  U1001   135040             1.0            1               1
690  U1001   135039             1.0            1               1
691  U1001   135045             1.0            1               1
692  U1001   135033             1.0            1               1
693  U1001   135025             2.0            2               2
694  U1001   135051             1.5            1               2
[
  {
    "placeID":132723,
    "place_score":0.6666666667,
    "Rcuisine":"Mexican"
  },
  {
    "placeID":132856,
    "place_score":0.2132007164,
    "Rcuisine":"Italian"
  },
  {
    "placeID":135041,
    "place_score":0.2132007164,
    "Rcuisine":"Bar"
  },
  {
    "placeID":135041,
    "place_score":0.2132007164,
    "Rcuisine":"Cafeteria"
  }

In [ ]:
#Test case to test the effiency of recommender engine

def take_userHistory():
  user_historyInput = input("Enter userID, placeID, overall_rating, food_rating, service_rating")
  testUser = user_historyInput.split()
  for i in testUser:
    testUser[1]= int(testUser[1])
    testUser[2] = float(testUser[2])
    testUser[3]= int(testUser[3])
    testUser[4] = int(testUser[4])
  #print(testUser)
  return testUser


inputs = []
def TestRecommender():
  n = int(input("Enter number of times user visited restaurant:"))
  for i in range(n):
    all_user_history = take_userHistory()
    inputs.append(all_user_history)

  df_test = pd.DataFrame(inputs, columns=['userID', 'placeID', 'overall_rating', 'food_rating', 'service_rating'])
  #print(df_test)
  data_final = pd.concat([data, df_test], ignore_index=True)
  return data_final
  
datafortest = TestRecommender()
#print(datafortest)
picked_userID=input("Enter User ID:")
specificUserData=datafortest.loc[datafortest['userID'] == picked_userID]
if (len(specificUserData.index)>2):  
    number_of_similar_user=100
    number_of_top_restaurants=100
    recommendList= recommendRestaurants(picked_userID,number_of_similar_user,number_of_top_restaurants)
    df1 = recommendList.rename(columns={'place':'placeID'})
    joined_frame = df1.merge(df, on='placeID', how='left')
    #Drop N/A
    joined_frame_without_NAN=joined_frame.dropna()
    result=joined_frame_without_NAN.drop('id', axis=1)
    print(result.to_json(orient='records', indent=2))
else:
    pRestaurants = recommendPopularRestaurants()
    # join with restaurants
    popCopyRestaurants = df.copy()
    # print(popCopyRestaurants)
    restaurants =  popCopyRestaurants.merge(pRestaurants, on="placeID", how="right")
    # sort values by asc
    restaurants = restaurants.sort_values(by='overall_rating', ascending=False)
    # drop the id column
    restaurants = restaurants.drop(['id'], axis=1)
    # convert to json response
    print(restaurants.to_json(orient='records', indent=2))

Enter number of times user visited restaurant:3
Enter userID, placeID, overall_rating, food_rating, service_ratingU1005 135038 1.3 2 3
Enter userID, placeID, overall_rating, food_rating, service_ratingU1005 132630 2 2 2
Enter userID, placeID, overall_rating, food_rating, service_ratingU1005 132663 1 1 1
U1005
[
  {
    "placeID":135054,
    "place_score":0.55,
    "Rcuisine":"Seafood"
  },
  {
    "placeID":135028,
    "place_score":0.3490969595,
    "Rcuisine":"Mexican"
  },
  {
    "placeID":135085,
    "place_score":0.234436782,
    "Rcuisine":"Fast_Food"
  },
  {
    "placeID":132872,
    "place_score":0.0909090909,
    "Rcuisine":"American"
  },
  {
    "placeID":132875,
    "place_score":0.0909090909,
    "Rcuisine":"Japanese"
  },
  {
    "placeID":132921,
    "place_score":0.0909090909,
    "Rcuisine":"Bar"
  },
  {
    "placeID":132925,
    "place_score":0.0909090909,
    "Rcuisine":"Mexican"
  },
  {
    "placeID":132937,
    "place_score":0.0909090909,
    "Rcuisine":"Bar"
 

# Todos
- [x] join operation on the recommended restaurants \
- [ ] group restaurants by id and then convert to json (maybe) \
- [x] return responses in json recommendRestaurants \
- [x] return responses in json recommendPopularRestaurants \
- [ ] take user input from the system \
- [ ] connect with the postgreSQL replica database \
- [ ] test data works correctly again \
- [ ] connect with the node application \
- [ ] intgrate backend with frontend application \
- [ ] test the whole application \